<a href="https://colab.research.google.com/github/suparuek2405/DADS6003_ML/blob/main/Multivariate_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Batch Gradient descent (Multiple linear regression)

In [7]:
import numpy as np
np.set_printoptions(precision=2)

In [34]:
def cost_function(theta, x, y, N):
  y_hat = x.dot(theta)
  c = (1/N)*np.sum((y_hat-y)**2)
  return c

In [9]:
def gradient_descent(alpha, x, y, ep=0.001, max_iter=10000):
  converged = False
  iter = 0
  N = x.shape[0] # number of samples
  print("Num of data = ",N)

  # initial theta
  theta =  np.random.random((x.shape[1],1))
  print("Init theta.shape = ",theta.shape)

  # total error, J(theta)
  J = cost_function(theta, x, y, N)
  print("First J = ",J)

  # Iterate Loop
  while not converged:

    y_hat = x.dot(theta)
    diff = y_hat - y
    grad = x.T.dot(diff)

    theta = theta - alpha * (1/N) * (grad)

    assert theta.shape == (3,1) #This line makes sure that the shape of theta is still be the same.

    # error
    J2 = cost_function(theta, x, y, N)

    if abs(J-J2) <= ep:
        print("       Converged, iterations: ", iter, "/", max_iter)
        converged = True

    J = J2   # update error s
    iter += 1  # update iter

    if iter == max_iter:
        print('       Max iterations exceeded!')
        converged = True

  #print("End converged iter = ",iter)
  return theta

In [10]:
if __name__ == '__main__':
  x = np.array([[0,1,],[2,6],[3,8]]) #x1, x2
  y = np.array([1,1,4])
  x_b = np.c_[np.ones((x.shape[0],1)),x]

  print("start main")
  print(x_b.shape)
  y = y.reshape(-1,1)
  print(y.shape)

  alpha = 0.01 # learning rate
  #Training process
  theta = gradient_descent(alpha, x_b, y, ep=0.00000000000001, max_iter=1000000)
  print ("Theta = ", theta)

  #predict trainned x
  xtest = np.array([[4,9]])
  xtest_b = np.c_[np.ones((xtest.shape[0],1)),xtest]
  y_p = xtest_b.dot(theta)
  print("y predict = ",y_p)

start main
(3, 3)
(3, 1)
Num of data =  3
Init theta.shape =  (3, 1)
First J =  4.632542903625601
       Converged, iterations:  363865 / 1000000
Theta =  [[ 7.]
 [15.]
 [-6.]]
y predict =  [[13.]]


## Stochastic GD from scratch


In [48]:
import numpy as np
def stochastic_gradient_descent(alpha, x, y, ep=0.001, max_iter=10000, decay_rate=0.01):
  converged = False
  iter = 0
  N = x.shape[0] # number of samples
  print("Num of data = ",N)

  # initial theta
  theta =  np.random.random((x.shape[1],1))
  print("Init theta.shape = ",theta.shape)

  # total error, J(theta)
  J = cost_function(theta, x, y, N)
  print("First J = ",J)

  # Iterate Loop
  while not converged:
    # Learning rate decay
    current_alpha = alpha / (1 + alpha*decay_rate * iter)

    permutation = np.random.permutation(N)
    x_shuffled = x[permutation]
    y_shuffled = y[permutation]
    for i in range(N):
      y_hat = x[i:i+1].dot(theta)
      diff = y_hat - y[i:i+1]
      grad = x[i:i+1].T.dot(diff)

      theta = theta - alpha * grad

      assert theta.shape == (3,1)

    # error
    J2 = cost_function(theta, x, y, N)

    if abs(J-J2) <= ep:
      print("       Converged, iterations: ", iter, "/", max_iter)
      print(f"       The last learning rate = {current_alpha}")
      converged = True

    J = J2
    iter += 1

    if iter == max_iter:
      print('       Max iterations exceeded!')
      converged = True

  return theta


In [49]:
if __name__ == '__main__':
  x = np.array([[0,1,],[2,6],[3,8]]) #x1, x2
  y = np.array([1,1,4])
  x_b = np.c_[np.ones((x.shape[0],1)),x]

  print("start main")
  print(x_b.shape)
  y = y.reshape(-1,1)
  print(y.shape)

  alpha = 0.01 # learning rate
  #Training process
  theta = stochastic_gradient_descent(alpha, x_b, y, ep=0.000000000001, max_iter=1000000, decay_rate=0.01)
  print ("Theta = ", theta)

  #predict trainned x
  xtest = np.array([[4,9]])
  xtest_b = np.c_[np.ones((xtest.shape[0],1)),xtest]
  y_p = xtest_b.dot(theta)
  print("y predict = ",y_p)

start main
(3, 3)
(3, 1)
Num of data =  3
Init theta.shape =  (3, 1)
First J =  2.596975524569722
       Converged, iterations:  76312 / 1000000
       The last learning rate = 0.0011585874501807397
Theta =  [[ 7.]
 [15.]
 [-6.]]
y predict =  [[13.]]


## Stochastic GD with SKlearn

In [81]:
from sklearn.linear_model import SGDRegressor

model = SGDRegressor(learning_rate='adaptive', eta0=0.01, penalty=None, max_iter=1000000, random_state=42)
x = np.array([[0,1,],[2,6],[3,8]]) #x1, x2
y = np.array([1,1,4])
model.fit(x, y)
print(model.intercept_," , ",model.coef_)

xtest = np.array([[4,9]])
y_p = model.predict(xtest)
print("y predict = ",y_p)

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, y)
print(model.intercept_, model.coef_)

[0.06]  ,  [0.14 0.33]
y predict =  [3.63]
6.999999999999989 [15. -6.]


## Mini-batch GD (size = 2)

In [80]:
import numpy as np

def minibatch_gradient_descent(alpha, x, y, batch_size=2, ep=0.001, max_iter=10000000, decay_rate=0.01):
    converged = False
    iter = 0
    N = x.shape[0]  # number of samples
    n_features = x.shape[1]  # number of features

    print("Num of data =", N)

    # initial theta (size of (n_features, 1))
    theta = np.random.random((n_features, 1))
    print("Init theta.shape =", theta.shape)

    # total error, J(theta)
    J = cost_function(theta, x, y, N)
    print("First J =", J)

    # Iterate Loop
    while not converged and iter < max_iter:
        # Learning rate decay
        current_alpha = alpha / (1 + alpha*decay_rate * iter)

        # Shuffle data
        permutation = np.random.permutation(N)
        x_shuffled = x[permutation]
        y_shuffled = y[permutation]

        # Loop over mini-batches
        for i in range(0, N, batch_size):
            x_batch = x_shuffled[i:i+batch_size]
            y_batch = y_shuffled[i:i+batch_size].reshape(-1, 1)

            # Make predictions
            y_hat = x_batch.dot(theta)
            diff = y_hat - y_batch

            # Calculate gradient
            grad = (x_batch.T.dot(diff)) / x_batch.shape[0]  # use the actual batch size

            # Update theta
            theta = theta - current_alpha * grad

        # Calculate new cost
        J2 = cost_function(theta, x, y, N)

        # Check for convergence
        if abs(J - J2) <= ep:
            print("       Converged, iterations:", iter, "/", max_iter)
            print(f"       The last learning rate = {current_alpha}")
            converged = True

        # Update for next iteration
        J = J2
        iter += 1

    if iter == max_iter:
        print('Max iterations exceeded!')

    return theta


In [73]:
if __name__ == '__main__':
  x = np.array([[0,1,],[2,6],[3,8]]) #x1, x2
  y = np.array([1,1,4])
  x_b = np.c_[np.ones((x.shape[0],1)),x]

  print("start main")
  print(x_b.shape)
  y = y.reshape(-1,1)
  print(y.shape)

  alpha = 0.01 # learning rate
  #Training process
  theta = minibatch_gradient_descent(alpha, x_b, y, batch_size=2, ep=0.0000000000001, max_iter=1000000, decay_rate=0.000001)
  print ("Theta = ", theta)

  #predict trainned x
  xtest = np.array([[4,9]])
  xtest_b = np.c_[np.ones((xtest.shape[0],1)),xtest]
  y_p = xtest_b.dot(theta)
  print("y predict = ",y_p)


start main
(3, 3)
(3, 1)
Num of data = 3
Init theta.shape = (3, 1)
First J = 1.2486834813701198
       Converged, iterations: 119211 / 1000000
       The last learning rate = 0.009988093094341304
Theta =  [[ 7.]
 [15.]
 [-6.]]
y predict =  [[13.]]
